In [42]:
import json
from dotenv import load_dotenv
from functools import reduce
import operator
import os
import pandas as pd
import pymongo
from pymongo import MongoClient
import requests

In [2]:
load_dotenv()

True

#### PARIS: Palais Garnier 

In [3]:
paris_ld = {'type': 'Point', 'coordinates': [48.87241163769417,2.3317031342524017]}

In [23]:
melbourne_cbd = {'type': 'Point', 'coordinates': [-37.81227175879301,144.96262186884945]}

In [24]:
sao_paulo_zc = {'type': 'Point', 'coordinates': [-23.533215960575276,-46.639408758375076]}

In [4]:
def getdata_fsq(url, city, venue):
    client_id = os.getenv("tok1")
    client_secret = os.getenv("tok2")
    url_ = f'{url}'
    c = city['coordinates'][0]
    c1 = city['coordinates'][1]
    parametros = {"client_id": client_id, "client_secret": client_secret, "v": "20180323", "ll": f'{c}, {c1}',
                  "query": venue, "limit": 55}
    resp = requests.get(url_, params=parametros).json()
    return resp

#### HACER UNA FUNCION PARA GENERAR DATAS POR CADA RUBRO, POR CADA CIUDAD

In [21]:
cathegories_p = ['vegan', 'airport', 'gare', 'starbucks', 'veterinaire', 'basketball', 'ecole']

In [22]:
def datagenerator(url, city, cathegories):
    cath = cathegories
    data_ = []
    for c in cath:
        data = getdata_fsq(url, city, c)
        data_.append(data['response']['venues'])
    return data_

In [23]:
paris = datagenerator('https://api.foursquare.com/v2/venues/search', paris_ld, cathegories_p)

In [ ]:
def getFromDict(dic,map_):
    return reduce(operator.getitem, map_, dic)

def type_point(gps):
    return {"type": "Point", "coordinates": gps}

def jsonxport(name, l):
    n = f'{name}.json'
    with open(n, 'w') as f:
        json.dump(l, f)

In [16]:
def jsongenerator(data, name):
    d = data
    map_name = ['name']
    map_categories = ['categories']
    map_lat = ['location', 'lat']
    map_long = ['location', 'lng']
    l = []
    for dic in d:
        ndic = {}
        ndic['name']= getFromDict(dic,map_name)
        ndic['categories']= getFromDict(dic,map_categories)
        ndic['latitude']= getFromDict(dic,map_lat)
        ndic['longitude']= getFromDict(dic,map_long)
        ndic['location']= type_point([ndic["latitude"],ndic["longitude"]])
        l.append(ndic)
    return jsonxport(name, l)

In [34]:
veg = jsongenerator(paris[0], 'veganp')

In [35]:
ap = jsongenerator(paris[1], 'airportp')

In [36]:
ga = jsongenerator(paris[2], 'gare')

In [37]:
sbs = jsongenerator(paris[3], 'starbucksp')

In [38]:
vet = jsongenerator(paris[4], 'veterinaire')

In [39]:
bas = jsongenerator(paris[5], 'basketball')

In [40]:
sc = jsongenerator(paris[6], 'schoolsp')

#### llamada a la collection

In [43]:
client = MongoClient("localhost:27017")
db = client.get_database("GeoSpatialProyect")

In [44]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'GeoSpatialProyect')

In [45]:
g = db.get_collection("transport")

In [50]:
g.find_one().keys()

dict_keys(['_id', 'name', 'categories', 'latitude', 'longitude', 'location'])

In [75]:
cond1 = {'categories.name': 'Airport'}

In [76]:
cond2 = {"location": {"$near": {"$geometry": paris_ld, "$minDistance": 0 ,"$maxDistance": 20000}}}

In [77]:
query = {'$and': [cond1, cond2]}

In [78]:
list(g.find(query))

[{'_id': ObjectId('6134a0313681de6a29cd9f4a'),
  'name': 'Aéroport Paris-Le Bourget (LBG)',
  'categories': [{'id': '4bf58dd8d48988d1ed931735',
    'name': 'Airport',
    'pluralName': 'Airports',
    'shortName': 'Airport',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/airport_',
     'suffix': '.png'},
    'primary': True}],
  'latitude': 48.95510050992192,
  'longitude': 2.4418187141418457,
  'location': {'type': 'Point',
   'coordinates': [48.95510050992192, 2.4418187141418457]}},
 {'_id': ObjectId('6134a0313681de6a29cd9f3b'),
  'name': 'Aéroport de Paris-Orly (ORY) (Aéroport de Paris-Orly)',
  'categories': [{'id': '4bf58dd8d48988d1ed931735',
    'name': 'Airport',
    'pluralName': 'Airports',
    'shortName': 'Airport',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/airport_',
     'suffix': '.png'},
    'primary': True}],
  'latitude': 48.72836947029652,
  'longitude': 2.3650217056274414,
  'location': {'type': 'Point',
   'coordin

In [283]:
#q1 = db.gastronomy.aggregate([{'$geoNear': {'near': paris_ld, 'distanceField': "distance", 'maxDistance': 1000, 'distanceMultiplier': 6371,
#                                   'spherical': True}}])